<center>
<img src="https://raw.githubusercontent.com/yoonkt200/python-data-analysis/master/img/this_is_data_anal.png" width="200" height="200"><br>
</center>

# ch02. 텍스트 마이닝 첫걸음



## 2.1 웹 크롤링으로 기초 데이터 수집하기


이번 절에서는 '나무위키 최근 변경 페이지'의 텍스트 데이터를 웹 크롤링으로 수집한 다음, 데이터 내에서 등장한 키워드의 출현 빈도를 분석해 보자.

### - 웹 크롤링 라이브러리 사용하기


파이썬에서는 BeautifulSoup과 requests라는 라이브러리 웹 크롤러를 만들 수 있다. requests는 특정 URL로부터 HTML 문서를 가져오는 작업을 수행하고, BeautifulSoup 모듈은 HTML 문서에서 데이터를 추출하는 작업을 수행한다.

- 페이지의 URL 정보 추출하기

In [1]:
# -*- coding: utf-8 -*-

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
import requests
from bs4 import BeautifulSoup
import re

# 크롤링할 사이트 주소를 정의한다.
source_url = "https://namu.wiki/RecentChanges"

# 사이트의 html 구조에 기반하여 크롤링을 수행합니다.
req = requests.get(source_url)
html = req.content
soup = BeautifulSoup(html, 'lxml')
contents_table = soup.find(name="table")
table_body = contents_table.find(name="tbody")
table_rows = table_body.find_all(name="tr")

In [4]:
# a태그의 href 속성을 리스트로 추출하여, 크롤링 할 페이지 리스트를 생성한다.
page_url_base = "https://namu.wiki"
page_urls = []
for index in range(0, len(table_rows)):
    first_td = table_rows[index].find_all('td')[0]
    td_url = first_td.find_all('a')
    if len(td_url) > 0:
        page_url = page_url_base + td_url[0].get('href')
        if 'png' not in page_url:
            page_urls.append(page_url)

# 중복 url을 제거한다.
page_urls = list(set(page_urls))
for page in page_urls[:5]:
    print(page)

https://namu.wiki/w/%EB%A6%AC%EB%B6%80%ED%8A%B8%20%EC%9B%94%EB%93%9C
https://namu.wiki/w/%EC%98%A4%EC%84%B8%EB%9D%BC%EB%B9%84
https://namu.wiki/w/%EB%85%B8%EC%B2%98%EB%85%80
https://namu.wiki/w/%EC%A0%84%EC%82%AC(%ED%95%98%EC%8A%A4%EC%8A%A4%ED%86%A4)/%EC%B9%B4%EB%93%9C%EC%9D%BC%EB%9E%8C/2020%EB%85%84
https://namu.wiki/w/%EC%88%9C%EA%B5%90


위 코드는 개발자 도구로 살펴본 HTML 구조에 기반하여 table->tbody->tr->td->a 태그 순의 HTML 계층 구조를 좁혀나가는 과정이다. 이 과정을 통해 목표 태그에 도달했을 때, get(href) 함수로 태그의 속성 정보를 추출한다. get()함수는 해당 태그가 가지고 있는 특정한 속성을 추출한다.

### - 텍스트 정보 수집하기


다음 코드는 최근 변경된 문서 중 한 페이지의 텍스트 정보를 크롤링한 것이다. 이전 단계와 다른 점은 get() 함수 대신 text() 함수를 사용하여 태그의 텍스트 정보만을 추출했다는 것이다.

In [5]:
req = requests.get(page_urls[0])
html = req.content
soup = BeautifulSoup(html, 'lxml')
contents_table = soup.find(name="article")
title = contents_table.find_all('h1')[0]
category = contents_table.find_all('ul')[0]
content_paragraphs = contents_table.find_all(name="div", attrs={"class":"wiki-paragraph"})
content_corpus_list = []

for paragraphs in content_paragraphs:
    content_corpus_list.append(paragraphs.text)
content_corpus = "".join(content_corpus_list)

print(title.text)
print("\n")
print(category.text)
print("\n")
print(content_corpus)

리부트 월드 


메이플스토리/시스템


1. 개요2. 특징2.1. 거래 불가2.2. 강화 시스템 간소화2.3. 패시브 스킬: 리부트2.4. 판매 아이템2.4.1. 잡화상점2.4.2. 캐시샵2.5. 몬스터들의 변화2.6. 기타 변화 요소3. 육성 팁4. 리부트 월드에서의 직업5. 역사5.1. 황선영 디렉터 시절5.2. 보돌팟 횡행 및 강렬한 힘의 결정 패치5.3. 공식홈페이지 랭킹 분리 이후5.4. 메이플스토리 고객간담회 이후6. 기록7. 문제점7.1. 현거래 문제7.2. 사냥터 자리 포화현상7.3. 특정 시점부터 과도하게 운에 의존하는 스펙업7.4. 리부트2 → 리부트 단일 방향 월드 리프8. 관련 문서 리부트월드는 유저간의 거래가 제한되며 캐릭터 성장의 재미가 더욱 강화된 월드입니다.2015년 7월 9일 리:부트 업데이트를 통해 등장한 메이플스토리의 특수 월드 시스템.일반 월드와는 클라이언트만 같을 뿐 사실상 다른 게임이라고 봐도 무방하다. 여러 커뮤니티에서 일반 월드와 리부트 월드는 서로 분리되어 있으며 유저들도 서로를 구분하고 있다. 특히 리부트 월드의 장점에 매력을 느껴 유입된 유저들도 많아지면서 이러한 분리 현상은 가속화되었다.V 업데이트 이후 동일한 시스템을 가진 '리부트2' 월드가 생기면서 리부트 월드는 고유 월드 특징을 가진 서버들을 일컫는 것으로 의미가 확장되었다. 기존의 리부트 월드는 구분을 위해 '리부트1', '리1'로, 리부트2는 '리2', '리부띠' 등으로 부르기도 한다.유저간 아이템 교환, 캐시교환, 경매장, 메소마켓을 포함한 모든 거래 시스템을 이용할 수 없다. 또한 아이템을 떨어뜨려도 아이템을 가지던 캐릭터가 아니면 그 아이템은 보이지 않는다. 단 유저가 직접 떨어뜨리지 않고 몬스터에게서 드랍되거나 채광, 채집을 통해 드랍된 아이템의 경우 드랍시킨 사람의 소유권이 없어지게 되면 다른 캐릭터에게도 보인다. 창고의 경우 월드 내 교환 가능한 소비 아이템과 메소, 전문기술에 사용되는 기타 아이템은 보관할 수 있다.하지만 장비 아이템은 

## 2.2 나무위키 최근 변경 페이지 키워드 분석하기


이제 분석에 사용할 데이터가 준비 되었으니, 본격적으로 텍스트 마이닝을 알아보자.

### step.1 크롤링: 웹 데이터 가져오기


이전 단계와 동일한 방법으로 웹 데이터를 크롤링한다. 단, 이번에는 모든 URL의 데이터를 가져오자.
다음 코드를 실행하여 나무위키에서 최근 변경이 일어난 페이지들의 URL을 page_urls라는 변수에 저장한다.

- 나무위키의 최근 변경 데이터 크롤링하기

In [6]:
# 크롤링한 데이터를 데이터 프레임으로 만들기 위해 준비한다.
columns = ['title', 'category', 'content_text']
df = pd.DataFrame(columns=columns)

# 각 페이지별 '제목', '카테고리', '본문' 정보를 데이터 프레임으로 만든다.
for page_url in page_urls:

    # 사이트의 html 구조에 기반하여 크롤링을 수행한다.
    req = requests.get(page_url)
    html = req.content
    soup = BeautifulSoup(html, 'lxml')
    contents_table = soup.find(name="article")
    title = contents_table.find_all('h1')[0]
    
    # 카테고리 정보가 없는 경우를 확인한다.
    if len(contents_table.find_all('ul')) > 0:
        category = contents_table.find_all('ul')[0]
    else:
        category = None
        
    content_paragraphs = contents_table.find_all(name="div", attrs={"class":"wiki-paragraph"})
    content_corpus_list = []
    
    # 페이지 내 제목 정보에서 개행 문자를 제거한 뒤 추출한다. 만약 없는 경우, 빈 문자열로 대체한다.
    if title is not None:
        row_title = title.text.replace("\n", " ")
    else:
        row_title = ""
    
    # 페이지 내 본문 정보에서 개행 문자를 제거한 뒤 추출한다. 만약 없는 경우, 빈 문자열로 대체한다.
    if content_paragraphs is not None:
        for paragraphs in content_paragraphs:
            if paragraphs is not None:
                content_corpus_list.append(paragraphs.text.replace("\n", " "))
            else:
                content_corpus_list.append("")
    else:
        content_corpus_list.append("")
        
    # 페이지 내 카테고리정보에서 “분류”라는 단어와 개행 문자를 제거한 뒤 추출한다. 만약 없는 경우, 빈 문자열로 대체한다.
    if category is not None:
        row_category = category.text.replace("\n", " ")
    else:
        row_category = ""
    
    # 모든 정보를 하나의 데이터 프레임에 저장한다.
    row = [row_title, row_category, "".join(content_corpus_list)]
    series = pd.Series(row, index=df.columns)
    df = df.append(series, ignore_index=True)

이제 이 주소들에 다시 한번 접근하여 문서의 본문과 제목, 그리고 카테고리에 등장하는 텍스트 데이터를 가져와보자.

In [7]:
df.head(5)

,title,category,content_text
0,리부트 월드,메이플스토리/시스템,1. 개요2. 특징2.1. 거래 불가2.2. 강화 시스템 간소화2.3. 패시브 스킬...
1,오세라비,1958년 출생부산광역시 출신 인물대한민국의 여성 정치인/ㅇ여성운동가한국 여성 작가...,가입 후 15일이 지나야 편집 가능한 문서입니다.이 문서는이 문단은 토론을 통...
2,노처녀,인물 특징사람 관련 한국어 단어,로그인 후 편집 가능한 문서입니다.이 문서는이 문단은 토론을 통해 다음의 합의...
3,전사(하스스톤)/카드일람/2020년,하스스톤/카드,하스스톤 카드일람정규전 카드 [ 열기 / 닫기 ]공용 카드특징일반등급희귀등급특급등급...
4,순교,종교 박해죽음,1. 개요2. 종교적 관점2.1. 기독교2.2. 이슬람2.3. 불교2.4. 그 외3...


위의 실행 결과는 모든 URL의 텍스트 데이터를 가져온 뒤, 이를 데이터 프레임의 형태로 변환한 것이다. 데이터에 등장하는 불필요한 문자인 '\n','분류'는 크롤링 과정에서 replace() 함수로 제거했다.

### step2 추출: 키워드 정보 추출하기


다음은 수집한 텍스트 데이터에서 키워드 정보를 추출하는 단계이다. 이를 위해 텍스트 전처리 작업이 필요하다. 텍스트 전처리는 특수문자나 외국어를 제거하는 등의 과정을 포함한다. 그런데 이는 언어와 상황마다 조금씩 다르다.
예를 들어 스팸메일을 분류하는 텍스트 마이닝의 경우, 특수문자나 외국어가 분석의 중요한 힌트가 될 수 있기 때문에 이를 제거하지 않는 편이다. 반면, 키워드 분석처럼 '단어'를 추출하는 것이 목적이라면 특정 언어의 글자만을 추출하기도 한다.
파이썬에서는 're'라는 모듈을 통해 정규표현식을 사용할 수 있다. 정규표현식이란 특정한 규칙을 가진 문자열의 집합을 표현하는 형식이다.

- 텍스트 데이터 전처리하기

In [9]:
# 텍스트 정제 함수 : 한글 이외의 문자는 전부 제거한다.
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글의 정규표현식을 나타낸다.
    result = hangul.sub('', text)
    return result

print(text_cleaning(df['content_text'][0]))

 개요 특징 거래 불가 강화 시스템 간소화 패시브 스킬 리부트 판매 아이템 잡화상점 캐시샵 몬스터들의 변화 기타 변화 요소 육성 팁 리부트 월드에서의 직업 역사 황선영 디렉터 시절 보돌팟 횡행 및 강렬한 힘의 결정 패치 공식홈페이지 랭킹 분리 이후 메이플스토리 고객간담회 이후 기록 문제점 현거래 문제 사냥터 자리 포화현상 특정 시점부터 과도하게 운에 의존하는 스펙업 리부트  리부트 단일 방향 월드 리프 관련 문서 리부트월드는 유저간의 거래가 제한되며 캐릭터 성장의 재미가 더욱 강화된 월드입니다년 월 일 리부트 업데이트를 통해 등장한 메이플스토리의 특수 월드 시스템일반 월드와는 클라이언트만 같을 뿐 사실상 다른 게임이라고 봐도 무방하다 여러 커뮤니티에서 일반 월드와 리부트 월드는 서로 분리되어 있으며 유저들도 서로를 구분하고 있다 특히 리부트 월드의 장점에 매력을 느껴 유입된 유저들도 많아지면서 이러한 분리 현상은 가속화되었다 업데이트 이후 동일한 시스템을 가진 리부트 월드가 생기면서 리부트 월드는 고유 월드 특징을 가진 서버들을 일컫는 것으로 의미가 확장되었다 기존의 리부트 월드는 구분을 위해 리부트 리로 리부트는 리 리부띠 등으로 부르기도 한다유저간 아이템 교환 캐시교환 경매장 메소마켓을 포함한 모든 거래 시스템을 이용할 수 없다 또한 아이템을 떨어뜨려도 아이템을 가지던 캐릭터가 아니면 그 아이템은 보이지 않는다 단 유저가 직접 떨어뜨리지 않고 몬스터에게서 드랍되거나 채광 채집을 통해 드랍된 아이템의 경우 드랍시킨 사람의 소유권이 없어지게 되면 다른 캐릭터에게도 보인다 창고의 경우 월드 내 교환 가능한 소비 아이템과 메소 전문기술에 사용되는 기타 아이템은 보관할 수 있다하지만 장비 아이템은 무슨 짓을 해도 교환할 수 없다 거래는 물론 창고 내에 보관하거나 땅에 떨어트려 놓고 주우러 가는 것도 불가능하다 자신의 캐릭터라도 다른 캐릭터에게 장비를 계승하지 못한다 이러한 시스템 때문에 리부트 월드에서는 모든 장비템을 직접 강화해야 한다 따라서 리부트 월드에서는 무

모든 데이터에 전처리를 적용하기 위해선 apply() 함수를 사용한다. 다음 코드는 title,category,content_text 3개의 피처에 apply() 함수를 적용한 것이다.

In [10]:
# 각 피처마다 데이터 전처리를 적용한다.
df['title'] = df['title'].apply(lambda x: text_cleaning(x))
df['category'] = df['category'].apply(lambda x: text_cleaning(x))
df['content_text'] = df['content_text'].apply(lambda x: text_cleaning(x))
df.head(5)

,title,category,content_text
0,리부트 월드,메이플스토리시스템,개요 특징 거래 불가 강화 시스템 간소화 패시브 스킬 리부트 판매 아이템 잡화상점...
1,오세라비,년 출생부산광역시 출신 인물대한민국의 여성 정치인ㅇ여성운동가한국 여성 작가평론가작가...,가입 후 일이 지나야 편집 가능한 문서입니다이 문서는이 문단은 토론을 통해 프로...
2,노처녀,인물 특징사람 관련 한국어 단어,로그인 후 편집 가능한 문서입니다이 문서는이 문단은 토론을 통해 다음의 합의사항...
3,전사하스스톤카드일람년,하스스톤카드,하스스톤 카드일람정규전 카드 열기 닫기 공용 카드특징일반등급희귀등급특급등급전설등...
4,순교,종교 박해죽음,개요 종교적 관점 기독교 이슬람 불교 그 외 세속적 관점 언어별 명칭 중국어 일본...


다음 과정은 키워드를 추출한 뒤, 빈도 분석을 수행하는 과정이다. 여기서 키워드를 추출한다는 것은 무엇을 의미할까? 키워드 추출이란 좁은 의미에서 명사, 혹은 형태소 단위의 문자열을 추출하는 것이다. 이를 수행하기 위해 말뭉치라는 것을 만들어야 한다.
말뭉치는 말 그대로 텍스트 데이터의 뭉텅이를 의미한다. 
다음 코드에서는 텍스트 피처를 tolist()로 추출한 뒤, join() 함수로 말뭉치를 생성했다.

- 말뭉치 만들기

In [11]:
# 각 피처마다 말뭉치를 생성한다.
title_corpus = "".join(df['title'].tolist())
category_corpus = "".join(df['category'].tolist())
content_corpus = "".join(df['content_text'].tolist())
print(title_corpus)

리부트 월드 오세라비 노처녀 전사하스스톤카드일람년 순교 돈 돈   눈보라유튜버 도수치료 명량 코야마 요시지 용문명일방주 졸프  킴블리 응원 월 일 여성시대 역고발 사태 물리치료 시바 카오루 머큐리얼 시리즈 이정미법조인 일원 주화 인천남동구민축구단역대 시즌  개발  다이노스년월 카스트 보스턴 대학교 던전앤파이터각성 히로인은 절망했습니다 아라가키 유이 나의 히어로 아카데미아등장인물 랭 랭글러 요통 스튜디오 유라원신 강구한 닌자닌자 슬레이어    박수영 박유화  성우극회 미나토 아쿠아 나리타 브라이언우마무스메   김가린 센서스튜디오 언어 차별 빌 터너 대 이상 나초인터넷 방송인 나나배우 분류노래 메필레스 더 다크 또봇 등장인물  전쟁전쟁의 영향 리터넌즈 캠페인 리그 오브 레전드아이템기본 파니팡  퓨처파이트업데이트 내역 에릭 다이어   카드 일람공화국 리코록맨  시리즈 비마엘 마친 블레이크 트라이넨 부엉이에듀등장인물 우르수스 포르쉐 한산 용의 출현 척추 가공의 언어 검은 교단던전 앤 파이터 한화 이글스년월 청주 버스  장원 번쩍번쩍 열매 이케조에 켄이치 중앙대학교학부경영경제대학 케이인터넷 방송인 토도로키 쇼토작중 행적 불경 국가별 사회 문제  캐릭터 제시 린가드 김태진방송인 


이제 각 말뭉치 안에서 등장하는 형태소를 추출하자. 파이썬의 한국어 형태소 추출 라이브러리 중 가장 쉽게 사용할 수 있는 konlpy를 사용한다.

다음 코드는 konlpy의 Okt tagger를 이용하여 형태소를 추출하는 과정이다. 그런데 추출 키워드를 자세히 보면 한 글자 키워드 혹은 실질적인 의미가 없는 키워드(분류,필요 등의 형태소)들을 발견할 수 있다. 이런 키워드는 어떻게 처리해야 할까?

- konlpy를 이용해 키워드 추출하기

In [13]:
from konlpy.tag import Okt
from collections import Counter

# konlpy의 형태소 분석기로 명사 단위의 키워드를 추출합니다.
nouns_tagger = Okt()
nouns = nouns_tagger.nouns(content_corpus)
count = Counter(nouns)
count

Counter({'개요': 66,
         '특징': 128,
         '거래': 33,
         '불가': 12,
         '강화': 173,
         '시스템': 127,
         '간소화': 11,
         '패': 140,
         '시브': 70,
         '스킬': 309,
         '리부트': 113,
         '판매': 88,
         '아이템': 208,
         '잡화': 6,
         '상점': 29,
         '캐시': 15,
         '몬스터': 82,
         '변화': 81,
         '기타': 78,
         '요소': 54,
         '육성': 37,
         '팁': 2,
         '월드': 151,
         '직업': 164,
         '역사': 1008,
         '황선영': 2,
         '디렉터': 10,
         '시절': 110,
         '보돌팟': 7,
         '횡행': 2,
         '및': 355,
         '힘': 199,
         '결정': 87,
         '패치': 138,
         '공식': 99,
         '홈페이지': 21,
         '랭킹': 23,
         '분리': 32,
         '이후': 667,
         '메이플스토리': 28,
         '고객': 11,
         '간담': 7,
         '회': 459,
         '기록': 272,
         '문제점': 48,
         '문제': 398,
         '사냥터': 14,
         '자리': 122,
         '포화': 13,
         '현상': 54,
         '특정': 52,
      

먼저 한글자 키워드를 제거하자. 특정한 의미를 가지는 한 글자 키워드는 따로 예외 처리를 해주는 것이 좋지만 이번엔 생략하자.

- 키워드 가다듬기 : 한 글자 키워드 제거하기

In [14]:
remove_char_counter = Counter({x:count[x] for x in count if len(x) > 1})
print(remove_char_counter)

Counter({'공격': 1406, '문서': 1144, '문단': 1067, '이전': 1066, '때문': 1013, '역사': 1008, '정도': 741, '경우': 740, '자신': 710, '사용': 693, '이후': 667, '사람': 657, '시간': 653, '토론': 632, '증가': 630, '사항': 626, '합의': 614, '한국': 577, '다른': 543, '피해': 522, '캐릭터': 501, '카드': 481, '유닛': 469, '시작': 466, '속도': 461, '감소': 459, '추가': 444, '모습': 437, '유저': 407, '문제': 398, '생각': 397, '대한': 379, '시부야': 379, '게임': 378, '아키바': 377, '닌자': 374, '상대': 371, '하나': 370, '대학교': 367, '등장': 362, '혜택': 361, '또한': 359, '위해': 358, '불이익': 348, '매우': 346, '레벨': 337, '기술': 336, '효과': 325, '시즌': 325, '전쟁': 322, '사실': 317, '기준': 315, '상황': 315, '당시': 310, '스킬': 309, '경기': 308, '부분': 306, '가장': 301, '업데이트': 296, '능력': 294, '이상': 293, '영화': 292, '다시': 291, '일본': 288, '유니폼': 287, '상태': 284, '등급': 284, '방송': 282, '현재': 280, '모두': 275, '빌드': 274, '기록': 272, '체력': 271, '국가': 266, '포르쉐': 264, '모든': 255, '편이': 254, '각성': 253, '수준': 249, '대해': 247, '가지': 246, '이름': 240, '대부분': 239, '미국': 238, '에너지': 238, '보고': 237, '선수': 237, '열매': 236, '지역': 

이제 실질적인 의미가 없는 키워드를 처리해보자. '입니다','그','저' 같은 관사나 접속사 등은 실질적인 의미가 없는 키워드이면서 동시에 의미적인 독립을 할 수 없는 품사이다. 이러한 키워드를 불용어라고 한다. 이 단계에서 필요한 것이 불용어 사전이다. 

- 키워드 가다듬기 : 불용어 사전

In [15]:
korean_stopwords_path = '/home/jaeyoon89/python-data-analysis/data/korean_stopwords.txt'

In [21]:
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]
print(stopwords[:10])

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라']


현재 분석중인 데이터에서만 적용되는 불용어도 추가해주어야 한다. '상위','문서','내용'과 같은 단어를 불용어 사전에 추가해 보자. 다음 코드는 새로운 불용어를 추가한 뒤, Counter 객체에서 불용어 리스트에 포함된 형태소를 제거하는 과정이다.

In [22]:
namu_wiki_stopwords = ['상위', '문서', '내용', '누설', '아래', '해당', '설명', '표기', '추가', '모든', '사용', '매우', '가장',
                       '줄거리', '요소', '상황', '편집', '틀', '경우', '때문', '모습', '정도', '이후', '사실', '생각', '인물', 
                       '이름', '년월']
for stopword in namu_wiki_stopwords:
    stopwords.append(stopword)

In [43]:
remove_char_counter = Counter({x : remove_char_counter[x] for x in count if x not in stopwords})
print(remove_char_counter)

Counter({'공격': 1406, '문단': 1067, '이전': 1066, '역사': 1008, '사람': 657, '토론': 632, '증가': 630, '사항': 626, '합의': 614, '한국': 577, '피해': 522, '캐릭터': 501, '카드': 481, '유닛': 469, '시작': 466, '속도': 461, '감소': 459, '유저': 407, '문제': 398, '대한': 379, '시부야': 379, '게임': 378, '아키바': 377, '닌자': 374, '상대': 371, '대학교': 367, '등장': 362, '혜택': 361, '위해': 358, '불이익': 348, '레벨': 337, '기술': 336, '효과': 325, '시즌': 325, '전쟁': 322, '기준': 315, '당시': 310, '스킬': 309, '경기': 308, '부분': 306, '업데이트': 296, '능력': 294, '영화': 292, '다시': 291, '일본': 288, '유니폼': 287, '상태': 284, '등급': 284, '방송': 282, '현재': 280, '빌드': 274, '기록': 272, '체력': 271, '국가': 266, '포르쉐': 264, '편이': 254, '각성': 253, '수준': 249, '대해': 247, '가지': 246, '대부분': 239, '미국': 238, '에너지': 238, '보고': 237, '선수': 237, '열매': 236, '지역': 232, '이동': 231, '기본': 229, '특성': 224, '카스트': 223, '다년': 219, '본인': 217, '태그': 213, '존재': 212, '전투': 210, '아이템': 208, '자체': 207, '이유': 207, '일부': 206, '처음': 205, '나라': 204, '선발': 203, '무기': 202, '사회': 199, '일반': 197, '실제': 196, '역시': 194, '수정': 

출처 : 이것이 데이터 분석이다 with 파이썬